# AP4 - Pattern Recognition
Implementation of Naive Bayes and Linear Discriminant Analysis for classification.

> Name: Jonas Carvalho Fortes

> Mat: 494513

## Load Datasets

In [15]:
import pandas as pd
from scipy.io import loadmat

dataset_input1 = loadmat('data/Input1.mat')
dataset_input1 = pd.DataFrame(dataset_input1['Input1']).transpose()
print(f'Dataset Input1 shape: {dataset_input1.shape}')

dataset_input2 = loadmat('data/Input2.mat')
dataset_input2 = pd.DataFrame(dataset_input2['Input2']).transpose()
print(f'Dataset Input2 shape: {dataset_input2.shape}')

Dataset Input1 shape: (4000, 2)
Dataset Input2 shape: (4000, 2)


In [18]:
print('Dataset Input1:')
dataset_input1

Dataset Input1:
Dataset Input2:


,0,1
0,0.686500,0.494410
1,0.760307,0.609878
2,0.053486,0.811601
3,1.173823,0.359015
4,0.328171,0.907802
...,...,...
3995,0.392458,0.208384
3996,-0.127486,-0.190746
3997,0.057968,-0.320305
3998,1.639673,0.866730


In [19]:
print('Dataset Input2:')
dataset_input2

Dataset Input2:


,0,1
0,0.686500,0.494410
1,0.760307,0.609878
2,0.053486,0.811601
3,1.173823,0.359015
4,0.328171,0.907802
...,...,...
3995,0.392458,0.208384
3996,-0.127486,-0.190746
3997,0.057968,-0.320305
3998,1.639673,0.866730
